In [ ]:
# Function: od_clf_cbf_qp_controller

## CLF-CBF-QP Controller
def od_clf_cbf_qp(t, x, R_gain, alpha, rho_0, p_rho, p_delta):
   # Define inputs
   sigma = x[0:3]
   w = x[3:6]
   hw = x[6:9]
   
   # Define desired parameters
   sigmad = np.array([0., 0., 0.])
   wd = np.array([0., 0., 0.])
   wdotd = np.array([0., 0., 0.])
   hwd = np.array([0., 0., 0.])
   
   # Define error dynamics
   sigmadc = np.array([[0., -sigmad[2], sigmad[1]], 
                     [sigmad[2], 0., -sigmad[0]], 
                     [-sigmad[1], sigmad[0], 0.]])
   
   sigmae = (sigmad * (sigma.T @ sigma - 1) + sigma * (1 - sigmad.T @ sigmad) - 2 * sigmadc @ sigma) / (1 + sigmad.T @ sigmad * sigma.T @ sigma - 2 * sigmad.T @ sigma)

   sigmaec = np.array([[0., -sigmae[2], sigmae[1]], 
                     [sigmae[2], 0., -sigmae[0]], 
                     [-sigmae[1], sigmae[0], 0.]])
   
   C = np.eye(3) + (8 * (sigmaec ** 2) - 4 * (1 - sigmae.T @ sigmae) * sigmaec) / ((1 + sigmae.T @ sigmae) ** 2)
   
   we = w - C @ wd
   
   wec = np.array([[0, -we[2], we[1]], 
                  [we[2], 0, -we[0]], 
                  [-we[1], we[0], 0]])
   
   he = hw - hwd
   
   # Define system parameters
   sigmac = np.array([[0., -sigma[2], sigma[1]], 
                     [sigma[2], 0., -sigma[0]], 
                     [-sigma[1], sigma[0], 0.]])
   
   M_sigmae = (1/4) * ((1 - (sigmae.T @ sigmae)) * np.eye(3) + 2 * sigmaec + 2 * (sigmae @ sigmae.T))
   
   sigmadote = M_sigmae @ we
   
   weCwd = we + C @ wd
   
   weCwdc = np.array([[0., -weCwd[2], weCwd[1]], 
                     [weCwd[2], 0., -weCwd[0]], 
                     [-weCwd[1], weCwd[0], 0.]])
   
   fx = np.concatenate([M_sigmae @ we,
      inv(J) @ ((-weCwdc @ (J @ weCwd + he)) + J @ (wec @ C @ wd - C @ wdotd)),
      np.zeros(3)])
   
   gx = np.concatenate([np.array([[0., 0., 0.], 
                     [0., 0., 0.], 
                     [0., 0., 0.]]),
                     inv(J),
                     -np.eye(3)])
   
   # Define Lie derivates
   eta = np.concatenate([sigmae, sigmadote])
   
   F = np.array([[0., 0., 0., 1., 0., 0.],
               [0., 0., 0., 0., 1., 0.],
               [0., 0., 0., 0., 0., 1.],
               [0., 0., 0., 0., 0., 0.],
               [0., 0., 0., 0., 0., 0.],
               [0., 0., 0., 0., 0., 0.]])
   
   G = np.array([[0., 0., 0.],
               [0., 0., 0.],
               [0., 0., 0.],
               [1., 0., 0.],
               [0., 1., 0.],
               [0., 0., 1.]])

   Q = np.eye(6)

   # Define the QP
   u = cp.Variable(3)
   rho = cp.Variable()
   delta = cp.Variable()
   
   sigmadotec = np.array([[0., -sigmadote[2], sigmadote[1]], 
                        [sigmadote[2], 0., -sigmadote[0]], 
                        [-sigmadote[1], sigmadote[0], 0.]])
   
   Mdot_sigmae = (1/4) * ((-2 * sigmae.T @ sigmadote) * np.eye(3) + 2 * sigmadotec + 2 * (sigmadote @ sigmae.T + sigmae @ sigmadote.T))
   
   L_f_2_h = Mdot_sigmae @ we + M_sigmae @ inv(J) @ (-weCwdc @ (J @ weCwd + he) + J @ (wec @ C @ wd - C @ wdotd))
   L_g_L_f_h = M_sigmae @ inv(J)
   
   L = inv(L_g_L_f_h)
   
   R = (L.T @ L) * R_gain

   P = ct.care(F, G, Q, R)[0]
   
   V = eta.T @ P @ eta
   
   L_fbar_V = eta.T @ (F.T @ P + P @ F) @ eta
   L_gbar_V = 2 * eta.T @ P @ G
   
   W = eta.T @ (Q + P @ G @ inv(R) @ G.T @ P) @ eta

   H = np.diag([1, 1, 1]) * 1

   umin = u_min * np.array([1., 1., 1.])
   umax = u_max * np.array([1., 1., 1.])
   
   ustar = -inv(L_g_L_f_h) @ L_f_2_h
   
   B_bar = h_max - hw
   B_unbar = hw - h_min
   
   # ############################# Optimal-Decay CLF-CBF-QP
   objective = cp.Minimize(cp.quad_form((L_g_L_f_h @ (u - ustar)), H) + p_delta * cp.square(delta) + p_rho * cp.square(1 - rho))
   constraints = [
                  L_fbar_V + L_gbar_V @ (L_g_L_f_h @ (u - ustar)) <= - rho * W + delta,
                  u >= -alpha * B_bar,
                  u <= alpha * B_unbar,
                  u >= umin,
                  u <= umax,
                  rho >= 0,
                  ]
   problem = cp.Problem(objective, constraints)
   problem.solve(solver=cp.PROXQP, verbose=False)
   
   return u.value, rho.value, delta.value
   # #############################